# **SSCS - Test Playground - tf.data**

Developed by André Paiva

Based on SSCS Dataset created by Helena Cuesta and Emilia Gómez

In [ ]:
EXECUTE_ON_COLAB = False

In [ ]:
import os
import pandas as pd
import h5py
import json
import time
import zipfile
import tables
import requests
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization

if(EXECUTE_ON_COLAB):
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
tf.config.list_physical_devices('GPU')

## 2 - Neural Network Model (VoasCNN)

In [ ]:
SAVE_MODEL = True
LOAD_MODEL = True
SPLIT_SIZE = 128
BATCH_SIZE = 32

In [ ]:
def voas_cnn_model():
    x_in = Input(shape=(360, SPLIT_SIZE, 1))
    
    x = BatchNormalization()(x_in)

    x = Conv2D(filters=32, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv1")(x)

    x = BatchNormalization()(x)

    x = Conv2D(filters=32, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv2")(x)

    x = BatchNormalization()(x)

    x = Conv2D(filters=16, kernel_size=(70, 3), padding="same",
        activation="relu", name="conv_harm_1")(x)

    x = BatchNormalization()(x)

    x = Conv2D(filters=16, kernel_size=(70, 3), padding="same",
        activation="relu", name="conv_harm_2")(x)

    ## start four branches now

    x = BatchNormalization()(x)

    ## branch 1
    x1a = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv1a")(x)

    x1a = BatchNormalization()(x1a)

    x1b = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv1b")(x1a)

    ## branch 2
    x2a = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv2a")(x)

    x2a = BatchNormalization()(x2a)

    x2b = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv2b")(x2a)

    ## branch 3

    x3a = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv3a")(x)

    x3a = BatchNormalization()(x3a)

    x3b = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv3b")(x3a)

    x4a = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv4a")(x)

    x4a = BatchNormalization()(x4a)

    x4b = Conv2D(filters=16, kernel_size=(3, 3), padding="same",
        activation="relu", name="conv4b"
    )(x4a)


    y1 = Conv2D(filters=1, kernel_size=1, name='conv_soprano',
                padding='same', activation='sigmoid')(x1b)
    y1 = tf.squeeze(y1, axis=-1, name='sop')
    y2 = Conv2D(filters=1, kernel_size=1, name='conv_alto',
                padding='same', activation='sigmoid')(x2b)
    y2 = tf.squeeze(y2, axis=-1, name='alt')
    y3 = Conv2D(filters=1, kernel_size=1, name='conv_tenor',
                padding='same', activation='sigmoid')(x3b)
    y3 = tf.squeeze(y3, axis=-1, name='ten')
    y4 = Conv2D(filters=1, kernel_size=1, name='conv_bass',
                padding='same', activation='sigmoid')(x4b)
    y4 = tf.squeeze(y4, axis=-1, name='bas')

    out = [y1, y2, y3, y4]

    model = Model(inputs=x_in, outputs=out, name='voasCNN')

    return model

## 3 - Auxiliar functions and Variables

Functions designed to manipulate the SSCS dataset.

### 3.1 - File path variables

In [ ]:
if(EXECUTE_ON_COLAB):
    dataset_dir = "/content/Datasets/"
    checkpoint_dir = "/content/drive/MyDrive/SSCS/Checkpoints/sscs.ckpt"
else:
    dataset_dir = "Datasets/"
    checkpoint_dir = "Checkpoints/sscs.ckpt"
zipname = dataset_dir + "SSCS_HDF5.zip"
sscs_dir = dataset_dir + "SSCS_HDF5/"

songs_dir = sscs_dir + "sscs/"
splitname = sscs_dir + "sscs_splits.json"

### 3.2 - Download/Extract Scripts

In [ ]:
def download(url, fname):
    
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    downloaded_size = 0
    with open(fname, 'wb') as file:
        for data in resp.iter_content(chunk_size=max(4096, int(total/10000))):
            size = file.write(data)
            downloaded_size += size
            percent = min(downloaded_size/total, 1.0)
            print(f"\r{percent:.2%} downloaded", end='')
            
    print()

In [ ]:
def sscs_download():
    
    if(not os.path.exists(dataset_dir)):
        os.mkdir(dataset_dir)
   
    if(not os.path.exists(zipname)):
        print("Downloading SSCS Dataset...")
        url = "https://github.com/Xornotor/SSCS_HDF5/releases/download/v1.0/SSCS_HDF5.zip"
        download(url, zipname)
    else:
        print("SSCS Dataset found.")

    if(not os.path.exists(sscs_dir)):
        print("Extracting SSCS Dataset...")
        with zipfile.ZipFile(zipname) as zf:
            os.mkdir(sscs_dir)
            zf.extractall(path=sscs_dir)
    else:
        print("SSCS Dataset already extracted.")
    
    print("Done.")

### 3.3 - Splits, songnames and songlists

In [ ]:
def sscs_get_split(split='train'):
    
    if(split.lower() == 'train' or split.lower() == 'validate' or
       split.lower() == 'test'):
        split_list = json.load(open(splitname, 'r'))[split.lower()]
        return split_list
    else:
        raise NameError("Split should be 'train', 'validate' or 'test'.")

In [ ]:
def sscs_pick_songlist(first=0, amount=5, split='train'):
    
    songnames = sscs_get_split(split)
    return songnames[first:first+amount]

In [ ]:
def sscs_pick_random_song(split='train'):
    
    songnames = sscs_get_split(split)
    rng = np.random.randint(0, len(songnames))
    return songnames[rng]

In [ ]:
def sscs_pick_multiple_random_songs(amount, split='train'):
    
    return [sscs_pick_random_song() for i in range(amount)]

### 3.4 - Plots

In [ ]:
def sscs_plot(dataframe):

    aspect_ratio = (3/8)*dataframe.shape[1]/dataframe.shape[0]
    fig, ax = plt.subplots(figsize=(13, 7))
    im = ax.imshow(dataframe, interpolation='nearest', aspect=aspect_ratio,
        cmap = mpl.colormaps['BuPu'])
    ax.invert_yaxis()
    plt.show()

## 4 - Download and extract dataset SSCS

In [ ]:
sscs_download()

## 5 - Dataset Generator

In [ ]:
class SSCS_Sequence(tf.keras.utils.Sequence):
    def __init__(self, filenames, batch_size=BATCH_SIZE):

        if(isinstance(filenames, np.ndarray)):
            self.filenames = [f.decode('utf-8') for f in filenames.tolist()]
        else:
            self.filenames = filenames

        self.batch_size = batch_size
        self.batches_amount = 0
        self.splits_amount = 0
        self.splits_per_file = np.array([], dtype=np.intc)
        self.songs_dir = songs_dir
        self.split_size = SPLIT_SIZE
        self.idx_get = np.array([], dtype=np.intc)
        self.split_get = np.array([], dtype=np.intc)
        self.debug = False

        for file in self.filenames:

            file_access = f"{self.songs_dir}{file}.h5"
            f = h5py.File(file_access, 'r')
            file_shape = f['mix/table'].shape[0]
            df_batch_items = file_shape//self.split_size
            #if(file_shape/self.split_size > df_batch_items): df_batch_items += 1
            self.splits_per_file = np.append(self.splits_per_file, int(df_batch_items))
            self.splits_amount += df_batch_items
            tmp_idx_get = np.array([self.filenames.index(file) for i in range(df_batch_items)], dtype=np.intc)
            tmp_split_get = np.array([i for i in range(df_batch_items)], dtype=np.intc)
            self.idx_get = np.append(self.idx_get, tmp_idx_get)
            self.split_get = np.append(self.split_get, tmp_split_get)
            f.close()
        
        self.batches_amount = self.split_get.shape[0]//self.batch_size
        if self.batches_amount < self.split_get.shape[0]/self.batch_size: 
            self.batches_amount += 1

        self.idx_get = np.resize(self.idx_get, self.batches_amount * self.batch_size)
        self.idx_get = np.reshape(self.idx_get, (-1, self.batch_size))

        self.split_get = np.resize(self.split_get, self.batches_amount * self.batch_size)
        self.split_get = np.reshape(self.split_get, (-1, self.batch_size))
     
    def __len__(self):

        return self.batches_amount

    def __getitem__(self, idx):

        if(self.debug):
            time_init = time.time()

        tmp_idx = self.idx_get[idx]
        tmp_split = self.split_get[idx]

        for i in range(self.batch_size):
            file_access = f"{self.songs_dir}{self.filenames[tmp_idx[i]]}.h5"
            data_min = tmp_split[i] * self.split_size
            data_max = data_min + self.split_size

            tmp_mix = np.transpose(pd.read_hdf(file_access, 'mix', start=data_min, stop=data_max)).to_numpy()
            tmp_s = np.transpose(pd.read_hdf(file_access, 'soprano', start=data_min, stop=data_max)).to_numpy()
            tmp_a = np.transpose(pd.read_hdf(file_access, 'alto', start=data_min, stop=data_max)).to_numpy()
            tmp_t = np.transpose(pd.read_hdf(file_access, 'tenor', start=data_min, stop=data_max)).to_numpy()
            tmp_b = np.transpose(pd.read_hdf(file_access, 'bass', start=data_min, stop=data_max)).to_numpy()

            tmp_mix = tmp_mix.reshape((1, tmp_mix.shape[0], tmp_mix.shape[1], 1))
            tmp_s = tmp_s.reshape((1, tmp_s.shape[0], tmp_s.shape[1], 1))
            tmp_a = tmp_a.reshape((1, tmp_a.shape[0], tmp_a.shape[1], 1))
            tmp_t = tmp_t.reshape((1, tmp_t.shape[0], tmp_t.shape[1], 1))
            tmp_b = tmp_b.reshape((1, tmp_b.shape[0], tmp_b.shape[1], 1))

            if(i == 0):
                mix_in = np.empty((0, tmp_mix.shape[1], tmp_mix.shape[2], tmp_mix.shape[3]), dtype=np.float32)
                s_out = np.empty((0, tmp_s.shape[1], tmp_s.shape[2], tmp_s.shape[3]), dtype=np.float32)
                a_out = np.empty((0, tmp_a.shape[1], tmp_a.shape[2], tmp_a.shape[3]), dtype=np.float32)
                t_out = np.empty((0, tmp_t.shape[1], tmp_t.shape[2], tmp_t.shape[3]), dtype=np.float32)
                b_out = np.empty((0, tmp_b.shape[1], tmp_b.shape[2], tmp_b.shape[3]), dtype=np.float32)

            mix_in = np.append(mix_in, tmp_mix, axis=0)
            s_out = np.append(s_out, tmp_s, axis=0)
            a_out = np.append(a_out, tmp_a, axis=0)
            t_out = np.append(t_out, tmp_t, axis=0)
            b_out = np.append(b_out, tmp_b, axis=0)

        mix_in = tf.convert_to_tensor(mix_in, dtype=tf.float32)
        s_out = tf.convert_to_tensor(s_out, dtype=tf.float32)
        a_out = tf.convert_to_tensor(a_out, dtype=tf.float32)
        t_out = tf.convert_to_tensor(t_out, dtype=tf.float32)
        b_out = tf.convert_to_tensor(b_out, dtype=tf.float32)

        if(self.debug):
            time_end = time.time()
            time_interval = time_end - time_init
            print(time_interval)

        return mix_in, (s_out, a_out, t_out, b_out)

    def get_splits_per_file(self):
        
        return self.splits_per_file

In [ ]:
seq = SSCS_Sequence(sscs_get_split()[30:60])

In [ ]:
seq.__len__()

In [ ]:
mix_test, satb_test = seq.__getitem__(2)
sscs_plot(mix_test.numpy()[23])

## 6 - Training VoasCNN

In [ ]:
dsSpec = tf.TensorSpec(shape=mix_test.shape, dtype=tf.float32)

signature = (dsSpec, (dsSpec, dsSpec, dsSpec, dsSpec))

ds = tf.data.Dataset.from_generator(SSCS_Sequence,
                                    args = [sscs_get_split()[30:60]],
                                    output_signature=signature
                                    ).cache().prefetch(tf.data.AUTOTUNE)
                                    #).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

#tf.data.DatasetSpec.from_value(ds)

In [ ]:
'''
class VoasCrossentropy(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        time_init = time.time()
        y_pred_sq = tf.squeeze(y_pred)
        y_true_sq = tf.squeeze(y_true)     
        elements =  -tf.math.multiply_no_nan(x=tf.math.log(y_pred_sq),
                                        y=y_true_sq) \
                    -tf.math.multiply_no_nan(x=tf.math.log(1 - y_pred_sq),
                                        y=(1 - y_true_sq))
        print(time.time()-time_init)
        return tf.reduce_mean(tf.reduce_sum(elements, range(tf.rank(y_pred_sq))))
'''

In [ ]:
voas_cnn = voas_cnn_model()
voas_cnn.compile(optimizer=Adam(learning_rate=5e-3),
                 loss=BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE),
                 metrics=[Precision()])
#voas_cnn.summary()

In [ ]:
'''
if(os.path.exists(checkpoint_dir)):
    voas_cnn.load_weights(checkpoint_dir)

save_cb = tf.keras.callbacks.ModelCheckpoint(   filepath=checkpoint_dir,
                                                save_weights_only=True,
                                                verbose=1
                                            )

voas_cnn.fit(ds, epochs=3, callbacks=[save_cb])
'''

In [ ]:
voas_cnn.fit(ds, epochs=10)

In [ ]:
mix, satb = seq.__getitem__(np.random.randint(0, seq.__len__() - 1))

mix = mix.numpy()

s = satb[0].numpy()
a = satb[1].numpy()
t = satb[2].numpy()
b = satb[3].numpy()

s_pred, a_pred, t_pred, b_pred = voas_cnn.predict(mix)

In [ ]:
idx = 10

sscs_plot(a[idx])
sscs_plot(a_pred[idx])
sscs_plot(mix[idx])